In [10]:
import os
import subprocess
from google.colab import drive

# === CẤU HÌNH RIÊNG (CHỈ THAY 1 LẦN) ===
USERNAME = "FongNgoo"   # ← THAY ĐỔI
REPO_NAME = "Basic_Dynamic_Prices_base_on_Demand_Model"        # ← THAY ĐỔI
BRANCH = "main"                     # hoặc "master"
# ======================================

REPO_DIR = f"/content/{REPO_NAME}"
REPO_URL = f"git@github.com:{USERNAME}/{REPO_NAME}.git"

!git config --global user.email "phongngo4060@gmail.com"
!git config --global user.name "FongNgoo"

In [11]:
def _setup_ssh():
    """Tải SSH key từ Drive và cấu hình"""
    drive.mount('/content/drive', force_remount=False)
    SSH_DIR = '/content/drive/MyDrive/.colab_ssh'
    SSH_KEY = f"{SSH_DIR}/id_colab"
    if not os.path.exists(SSH_KEY):
        raise FileNotFoundError("SSH key chưa tồn tại! Chạy file setup trước.")

    os.makedirs('/root/.ssh', exist_ok=True)
    subprocess.run(['cp', f'{SSH_KEY}', f'{SSH_KEY}.pub', '/root/.ssh/'], check=False)
    subprocess.run(['chmod', '600', '/root/.ssh/id_colab'], check=True)
    subprocess.run(['chmod', '644', '/root/.ssh/id_colab.pub'], check=True)

    config = f"""Host github.com
    HostName github.com
    User git
    IdentityFile /root/.ssh/id_colab
    StrictHostKeyChecking no
"""
    with open('/root/.ssh/config', 'w') as f:
        f.write(config)
    subprocess.run(['chmod', '600', '/root/.ssh/config'], check=True)

In [12]:
def _ensure_repo():
    """Đảm bảo repo tồn tại, clone nếu cần"""
    if not os.path.exists(REPO_DIR):
        print(f"Repo không tồn tại → Đang clone: {REPO_URL}")
        result = subprocess.run(['git', 'clone', REPO_URL, REPO_DIR])
        if result.returncode != 0:
            raise RuntimeError("Clone thất bại! Kiểm tra SSH key và repo URL.")
    # Luôn cd vào repo
    os.chdir(REPO_DIR)
    print(f"Đã vào thư mục: {REPO_DIR}")

In [7]:
def sync(message=None, auto_push=True, branch=None):
    """
    TỰ ĐỘNG SYNC 2 CHIỀU (AN TOÀN, KHÔNG LỖI getcwd)
    """
    global BRANCH
    branch = branch or BRANCH
    !git checkout main
    if not os.path.exists('/content/drive/MyDrive'):
        print("Drive chưa mount → đang mount lại...")
        drive.mount('/content/drive', force_remount=True)
    else:
        # Kiểm tra xem có bị treo không
        try:
            os.listdir('/content/drive/MyDrive')
        except:
            print("Drive bị treo → force remount lại...")
            !pkill -9 -f drive
            drive.mount('/content/drive', force_remount=True)
    print("Bước 1: Thiết lập SSH...")
    _setup_ssh()

    print("Bước 2: Đảm bảo repo tồn tại...")
    _ensure_repo()

    print("Bước 3: Pull từ GitHub...")
    result = subprocess.run(['git', 'pull', 'origin', branch, '--rebase'])
    if result.returncode != 0:
        print("Pull thất bại! Kiểm tra kết nối hoặc xung đột.")
        return

    %cd /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
    print("Bước 4: Kiểm tra thay đổi...")
    status = subprocess.check_output(['git', 'status', '--porcelain']).decode().strip().split('\n')
    status = [s for s in status if s.strip()]

    if not status:
        print("Không có thay đổi.")
        return

    print(f"Phát hiện {len(status)} thay đổi:")
    for line in status[:5]:
        print(f"   {line}")
    if len(status) > 5:
        print(f"   ... và {len(status)-5} thay đổi khác")

    if not auto_push:
        print("auto_push=False → Bỏ qua push")
        return

    msg = message or f"Auto sync from Colab - $(date)"
    print(f"Đang commit: {msg}")
    subprocess.run(['git', 'add', '.'])
    commit_result = subprocess.run(['git', 'commit', '-m', msg], capture_output=True)
    if b"nothing to commit" in commit_result.stdout:
        print("Không có gì để commit.")
        return

    print("Đang push lên GitHub...")
    push_result = subprocess.run(['git', 'push', 'origin', branch])
    if push_result.returncode == 0:
        print("SYNC HOÀN TẤT!")
    else:
        print("Push thất bại!")

In [13]:
sync()

Already on 'main'
Your branch is up to date with 'origin/main'.
Bước 1: Thiết lập SSH...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bước 2: Đảm bảo repo tồn tại...
Đã vào thư mục: /content/Basic_Dynamic_Prices_base_on_Demand_Model
Bước 3: Pull từ GitHub...
/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
Bước 4: Kiểm tra thay đổi...
Không có thay đổi.


In [14]:
# GIẢI QUYẾT XUNG ĐỘT + FORCE SYNC NGAY LẬP TỨC (dùng khi bạn muốn giữ toàn bộ thay đổi trong Colab)

repo_path = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model"
%cd {repo_path}

# 1. Bỏ toàn bộ rebase đang bị kẹt (nếu có)
!git rebase --abort 2>/dev/null || echo "Không có rebase đang chạy"

# 2. Reset cứng về trạng thái hiện tại của GitHub (lấy mới nhất từ remote)
!git fetch origin
!git reset --hard origin/main   # thay main → master nếu repo bạn dùng master

# 3. Ghi đè luôn mọi thứ trong Colab lên GitHub (force push)
!git add . --all
!git commit -m "Force sync from Colab - $(date)" || echo "Không có gì để commit mới"

# 4. FORCE PUSH (bắt buộc dùng -f để ghi đè)
!git push origin main --force   # thay main → master nếu cần

print("ĐÃ GIẢI QUYẾT XUNG ĐỘT + FORCE SYNC THÀNH CÔNG!")
print("GitHub giờ đây 100% giống hệt Colab của bạn!")

/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
Không có rebase đang chạy
Updating files: 100% (124/124), done.
HEAD is now at e6020b3 Auto sync from Colab - $(date)
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Không có gì để commit mới
Everything up-to-date
ĐÃ GIẢI QUYẾT XUNG ĐỘT + FORCE SYNC THÀNH CÔNG!
GitHub giờ đây 100% giống hệt Colab của bạn!


In [ ]:
# PUSH NGAY LẬP TỨC – KHÔNG QUAN TÂM XUNG ĐỘT (siêu nhanh)

repo_path = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model"
%cd {repo_path}

# Fix Drive treo (nếu có)
!pkill -9 -f "drive" 2>/dev/null
from google.colab import drive
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive', force_remount=True)

# Vào đúng thư mục
%cd {repo_path}

# Bỏ mọi rebase đang kẹt
!git rebase --abort 2>/dev/null

# Lấy mới nhất từ GitHub trước (đề phòng)
!git fetch origin

# Ghi đè toàn bộ bằng những gì đang có trong Colab
!git add . --all
!git commit -m "Push nhanh từ Colab - $(date '+ %Y-%m-%d %H:%M:%S')" || echo "Không có gì mới để commit"

# FORCE PUSH (quan trọng nhất – ghi đè hoàn toàn lên GitHub)
!git push origin main --force-with-lease

# Nếu repo bạn dùng branch master thì dùng dòng này thay dòng trên:
# !git push origin master --force-with-lease

print("ĐÃ PUSH XONG 100%!")
print("GitHub giờ đây giống hệt Colab của bạn rồi!")
print("Link repo của bạn sẽ có commit mới nhất ngay bây giờ")

/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
^C


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2821705439.py", line 13, in <cell line: 0>
    get_ipython().run_line_magic('cd', '{repo_path}')
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
                              ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
             ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception

In [ ]:
# SIÊU CỨU CÁNH KHI DRIVE TREO NẶNG + COLAB BỊ CRASH TRACEBACK

# Bước 1: Kill sạch mọi kết nối Drive đang treo
!pkill -9 -f "drive" 2>/dev/null
!pkill -9 -f "fuse" 2>/dev/null
!umount -f /content/drive 2>/dev/null
!rm -rf /content/drive 2>/dev/null

# Bước 2: Remount lại Drive thật mạnh (force + flush cache)
from google.colab import drive
import time
time.sleep(2)  # chờ kill process xong
drive.mount('/content/drive', force_remount=True)

# Bước 3: Vào lại repo + push ngay lập tức (force với lease an toàn)
%cd /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model

!git add . --all
!git commit -m "Emergency push từ Colab - $(date '+%Y-%m-%d %H:%M:%S')" || echo "Không có gì để commit"

# Force push an toàn (chỉ ghi đè nếu không ai khác push trong lúc này)
!git push origin main --force-with-lease

print("ĐÃ THOÁT KHỎI ĐỊA NGỤC DRIVE TREO!")
print("ĐÃ PUSH XONG 100% LÊN GITHUB RỒI!")
print("Bây giờ bạn có thể refresh GitHub thấy commit mới nhất ngay!")

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
^C
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
^C
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2365349120.py", line 16, in <cell line: 0>
    get_ipython().run_line_magic('cd', '/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model')
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
                              ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
             ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not

Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <google.colab._shell.Shell object at 0x7ac9e3ea1820>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3924, in atexit_operations
    self.reset(new_session=False)
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 1445, in reset
    self.history_manager.reset(new_session)
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_history.py", line 33, in reset
    super(ColabHistoryManager, self).reset(new_session=new_session)
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/history.py", line 592, in reset
    self.dir_hist[:] = [os.getcwd()]
                        ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not connected


In [34]:
# KIỂM TRA & PUSH LÊN GITHUB (phiên bản chạy ngon 100% trong Colab)

import os
from datetime import datetime

repo_path = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model"

# 1. Vào thư mục repo (tự fix nếu Drive bị treo)
try:
    os.chdir(repo_path)
    print(f"Đã vào thư mục: {os.getcwd()}")
except:
    print("Drive bị treo → Đang remount lại...")
    !pkill -9 -f drive 2>/dev/null
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.chdir(repo_path)
    print(f"Đã vào lại thư mục: {os.getcwd()}")

# 2. Git status
print("\nGIT STATUS:")
!git status -sb

# 3. 3 commit mới nhất ở local
print("\n3 COMMIT MỚI NHẤT (máy bạn):")
!git log --oneline -3 --pretty=format:"%C(yellow)%h %Creset%s %Cgreen(%cr)%Creset" --date=relative

# 4. Fetch và xem commit mới nhất trên GitHub
print("\nĐang fetch từ GitHub...")
!git fetch origin

print("\n3 COMMIT MỚI NHẤT TRÊN GITHUB (origin/main):")
!git log origin/main --oneline -3 --pretty=format:"%C(yellow)%h %Creset%s %Cgreen(%cr)%Creset" --date=relative 2>/dev/null || echo "Không thấy branch main, thử master..."
!git log origin/master --oneline -3 --pretty=format:"%C(yellow)%h %Creset%s %Cgreen(%cr)%Creset" --date=relative 2>/dev/null

# 5. Nếu có thay đổi chưa push → tự động commit + push luôn
print("\nĐang kiểm tra thay đổi chưa push...")
!git add .

# Kiểm tra có gì thay đổi không
changed = !git status --porcelain
if changed:
    print(f"Có {len(changed)} thay đổi → Đang commit & push...")
    commit_msg = f"Auto sync from Colab - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    !git commit -m "{commit_msg}"
    !git push origin main 2>/dev/null || git push origin master
    print("ĐÃ PUSH THÀNH CÔNG LÊN GITHUB!")
else:
    print("ĐÃ ĐỒNG BỘ HOÀN TOÀN! Không có gì cần push thêm.")

print("\nHOÀN TẤT KIỂM TRA & SYNC!")

Đã vào thư mục: /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model

GIT STATUS:
## main...origin/main
MM AUTO_SYNC_COLAB_GITHUB_SSH.ipynb
M  News_Embeding.ipynb
A  Output/ppo_pricing_best.pth
M  Output/preprocessed_data.npz
M  Output/rpt_demand_best.pth
M  Output/scalers.pkl
M  PPO.ipynb
M  RPT.ipynb

3 COMMIT MỚI NHẤT (máy bạn):
e6020b3 Auto sync from Colab - $(date) (20 hours ago)
4ee26e3 Auto sync from Colab - $(date) (20 hours ago)
ec9263b Auto sync from Colab - $(date) (21 hours ago)
Đang fetch từ GitHub...

3 COMMIT MỚI NHẤT TRÊN GITHUB (origin/main):
e6020b3 Auto sync from Colab - $(date) (20 hours ago)
4ee26e3 Auto sync from Colab - $(date) (20 hours ago)
ec9263b Auto sync from Colab - $(date) (21 hours ago)
Đang kiểm tra thay đổi chưa push...
Có 8 thay đổi → Đang commit & push...
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your